In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [42]:
symbols_path = './backbone/data/backtest/symbols/SP500_H1.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2012-01-16 00:00:00,1290.900,1291.100,1288.100,1288.100,0.0000
1,2012-01-16 01:00:00,1287.900,1288.400,1286.400,1287.400,0.0000
2,2012-01-16 02:00:00,1287.600,1287.600,1286.600,1287.100,0.0000
3,2012-01-16 03:00:00,1286.900,1288.400,1286.900,1287.100,0.0000
4,2012-01-16 04:00:00,1286.900,1287.100,1286.100,1286.100,0.0000
...,...,...,...,...,...,...
63251,2024-08-30 16:00:00,5600.337,5605.937,5581.014,5600.354,29.2410
63252,2024-08-30 17:00:00,5600.116,5607.377,5595.127,5597.849,20.0450
63253,2024-08-30 18:00:00,5597.970,5618.452,5595.220,5616.233,14.5300
63254,2024-08-30 19:00:00,5615.983,5651.087,5610.517,5645.139,23.4720


In [43]:
# from datetime import datetime
# import MetaTrader5 as mt5
# import pytz

# print("MetaTrader5 package author: ", mt5.__author__)
# print("MetaTrader5 package version: ", mt5.__version__)

# # establish connection to MetaTrader 5 terminal
# if not mt5.initialize():
#     raise Exception("initialize() failed, error code =",mt5.last_error())

# # set time zone to UTC
# timezone = pytz.timezone("Etc/UTC")

# # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
# utc_from = datetime(2000, 5, 1, tzinfo=timezone)
# utc_to = datetime(2024, 1, 1, tzinfo=timezone)
# rates = mt5.copy_rates_range('US500m', mt5.TIMEFRAME_H3, utc_from, utc_to)

# mt5.shutdown()

# # create DataFrame out of the obtained data
# df = pd.DataFrame(rates)

# # convert time in seconds into the datetime format
# df['time'] = pd.to_datetime(df['time'], unit='s')
                          
# df = df.rename(columns={
#   'time':'Date', 
#   'open':'Open', 
#   'high':'High', 
#   'low':'Low', 
#   'close':'Close', 
#   'tick_volume':'Volume'
# })

# df

In [44]:
train_start = '2012-01-01'
train_end = '2016-01-01'

test_start = '2016-01-01'
test_end = '2019-01-01'

wfo_start = '2019-01-01'
wfo_end = '2024-01-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


In [45]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


In [51]:

import itertools

class LowerLow(Strategy):
    pip_size = 0.0001
    risk=5
    n=5
    
    def init(self):
        self.daily_sma_200 = resample_apply(
            'D', ta.SMA, self.data.Close, timeperiod=200
        )

        self.daily_sma_5 = resample_apply(
            'D', ta.SMA, self.data.Close, timeperiod=5
        )

        self.daily_rsi_2 = resample_apply(
            'D', ta.RSI, self.data.Close, 2
        )
        
        self.daily_ll, self.daily_hh = resample_apply(
            'D', ll_hh_indicator, self.data.Close, window=10
        )

        self.ll, self.hh = self.I(
            ll_hh_indicator, self.data.Close, window=5
        )

    
    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.daily_rsi_2 >= 65 or time_in_position > 5:
                    self.position.close()

        else:
            cum_rsi = self.daily_rsi_2[-1] + self.daily_rsi_2[-2]

            if actual_close > self.daily_sma_200 and self.daily_ll and cum_rsi <= 35 and self.ll:
                self.buy(size=self.risk/100)
                self.random_time = 5


bt_train = Backtest(
    train_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()
stats

Start                     2012-01-16 00:00:00
End                       2015-12-31 17:00:00
Duration                   1445 days 17:00:00
Exposure Time [%]                   10.616803
Equity Final [$]                 18922.691548
Equity Peak [$]                  19157.406548
Return [%]                          26.151277
Buy & Hold Return [%]               59.737753
Return (Ann.) [%]                    5.554359
Volatility (Ann.) [%]                6.525433
Sharpe Ratio                         0.851186
Sortino Ratio                        1.502639
Calmar Ratio                         0.884908
Max. Drawdown [%]                   -6.276767
Avg. Drawdown [%]                   -0.736023
Max. Drawdown Duration      244 days 05:00:00
Avg. Drawdown Duration       16 days 16:00:00
# Trades                                   36
Win Rate [%]                        72.222222
Best Trade [%]                       2.754922
Worst Trade [%]                      -2.48864
Avg. Trade [%]                    

In [52]:
bt_train.plot(filename='./RsiBBands.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p15300', ...)

In [53]:
bt_train = Backtest(
    test_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2016-01-04 07:00:00
End                       2018-12-31 21:00:00
Duration                   1092 days 14:00:00
Exposure Time [%]                    8.099084
Equity Final [$]                 14467.527022
Equity Peak [$]                  16245.002373
Return [%]                           -3.54982
Buy & Hold Return [%]               24.131414
Return (Ann.) [%]                    -1.11543
Volatility (Ann.) [%]                7.013011
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.966347
Avg. Drawdown [%]                   -0.805285
Max. Drawdown Duration      329 days 06:00:00
Avg. Drawdown Duration       24 days 09:00:00
# Trades                                   23
Win Rate [%]                        69.565217
Best Trade [%]                       2.122286
Worst Trade [%]                      -5.19413
Avg. Trade [%]                    

# WFO

In [ ]:
import pickle
from wfo_utils.utils import walk_forward, plot_full_equity_curve, plot_stats

In [ ]:

lookback_bars = 365 * 24
validation_bars = 210 * 24
warmup_bars = 200 * 24 

params = {
    'n': [10],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    LowerLow,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

In [ ]:
for x in range(0, len(stats)):
    plot_stats(df, stats[x], LowerLow, plot=True)

In [ ]:
plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
